## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [13]:
# Import all required libraries

import numpy as np
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier

In [14]:
train_datasets = np.array([34, 7, 53])
test_datasets = np.array([27])

In [15]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


# Helper function to load dataset
def load_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_train/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_dataset("B", dataset) for dataset in train_datasets])
    labels = np.concatenate([load_dataset_labels("B", dataset) for dataset in train_datasets])
    return dataset, labels


def get_test_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_test_dataset("B", dataset) for dataset in test_datasets])
    labels = np.concatenate([load_test_dataset_labels("B", dataset) for dataset in test_datasets])
    return dataset, labels

In [16]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=1762,
    T=4365,
    s=63.0,
    max_included_literals=666,
    weighted_clauses=True,
    type_iii_feedback=False,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    accuracy = np.sum(pred == y_test) / len(y_test)

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    print(f"Accuracy = {accuracy:.5f}, TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}")

    save_accuracy(epoch, accuracy, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {accuracy:.5f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy

        print("Saving model")
        save_model(tm, "best_1.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs
Predicted 0s: 5989, Predicted 1s: 3948
Accuracy = 0.46292, TP = 3698, TN = 902, FP = 250, FN = 5087
Epoch: 1, Accuracy: 0.46292
Saving model
Predicted 0s: 5959, Predicted 1s: 3978
Accuracy = 0.46755, TP = 3736, TN = 910, FP = 242, FN = 5049
Epoch: 2, Accuracy: 0.46755
Saving model
Predicted 0s: 6170, Predicted 1s: 3767
Accuracy = 0.44450, TP = 3516, TN = 901, FP = 251, FN = 5269
Epoch: 3, Accuracy: 0.44450
Predicted 0s: 5943, Predicted 1s: 3994
Accuracy = 0.46714, TP = 3742, TN = 900, FP = 252, FN = 5043
Epoch: 4, Accuracy: 0.46714
Predicted 0s: 5895, Predicted 1s: 4042
Accuracy = 0.47157, TP = 3788, TN = 898, FP = 254, FN = 4997
Epoch: 5, Accuracy: 0.47157
Saving model
Predicted 0s: 5911, Predicted 1s: 4026
Accuracy = 0.47056, TP = 3775, TN = 901, FP = 251, FN = 5010
Epoch: 6, Accuracy: 0.47056
Predicted 0s: 6001, Predicted 1s: 3936
Accuracy = 0.46110, TP = 3683, TN = 899, FP = 253, FN = 5102
Ep